In [ ]:
import json
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

### Constants and hyperparameters

In [ ]:
epochs = 500 # Number of training epochs
batch_size = 16 # Batch size
split_percentage = 0.8 # Training and test set splitting percentage
validation_split = 0.2 # Validation set percentage
early_stopping_patience = 15 # Number of epochs of patience before triggering early stopping

In [ ]:
# CHANGE ME
dataset_path = "../Dataset/Flow signals/ND_signals/3d_flow_signals_1.json" # Dataset path
flow_quantity = "p" # Flow quantity to be used as feature

### Data loading

In [ ]:
naca_numbers = ['maximum_camber', 'maximum_camber_position', 'maximum_thickness']

dataset = []
with open(dataset_path, 'r') as dataset_file:
  samples = json.load(dataset_file)
  for sample in samples:
    dataset.append({
        "features": sample["features"][flow_quantity],
        "labels": list(sample["naca_numbers"].values())
    })

### Shuffling the dataset

In [ ]:
# Shuffling the dataset
np.random.shuffle(dataset)

### Training and test set

In [ ]:
# Computing the number of training samples according to the splitting percentage
num_training_samples = int(np.floor(split_percentage * len(dataset)))

# Extracting the training and test set
training_set, test_set = dataset[:num_training_samples], dataset[num_training_samples:]

In [ ]:
# Extracting the training features and labels
train_features = np.array([sample["features"] for sample in training_set])
train_labels = np.array([sample["labels"] for sample in training_set])

# Extracting the test features and labels
test_features = np.array([sample["features"] for sample in test_set])
test_labels = np.array([sample["labels"] for sample in test_set])

### Data normalization

In [ ]:
# Computing the mean and standard deviation of the training features
mean = train_features.mean(axis=0)
std = train_features.std(axis=0)

In [ ]:
# Normalizing the training and test features w.r.t. the training statistics
normalized_train_features = (train_features - mean) / std
normalized_test_features = (test_features - mean) / std

In [ ]:
# Plotting a random sample
plt.plot(normalized_train_features[random.choice([0, len(normalized_train_features)-1])])
plt.show()

### Building the model

In [ ]:
# BEST MODEL FOR 1D SIGNALS
def buildModel():
  # Sequential model - CNN 1D
  model = keras.Sequential([
    keras.layers.InputLayer(input_shape=np.shape(normalized_train_features)[1:]),
    keras.layers.Conv1D(filters=30, kernel_size=3, activation=tf.nn.tanh),
    keras.layers.AveragePooling1D(pool_size=2),
    keras.layers.Dropout(0.01),
    keras.layers.Conv1D(filters=20, kernel_size=3, activation=tf.nn.tanh),
    keras.layers.AveragePooling1D(pool_size=2),
    keras.layers.Dropout(0.01),
    keras.layers.Flatten(),
    keras.layers.Dense(20, activation=tf.nn.tanh),
    keras.layers.Dense(10, activation=tf.nn.tanh),
    keras.layers.Dense(len(naca_numbers))
  ])

  # Compiling the model
  model.compile(loss='mse', optimizer='adam', metrics=['mae'])
  
  return model

In [ ]:
model = buildModel()
model.summary()

### Model training

In [ ]:
# Early stopping with a predefined patience
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=early_stopping_patience)

# Training the model
history = model.fit(
    normalized_train_features, 
    train_labels,
    epochs=epochs,
    validation_split=validation_split,
    verbose=1,
    callbacks=[early_stopping]
)

In [ ]:
# Function to plot the metrics of training and validation
def plotHistory(history, training_metric, validation_metric, ylabel):
  plt.plot(history.history[training_metric], label=training_metric)
  plt.plot(history.history[validation_metric], label=validation_metric)
  plt.ylim([0, np.max(history.history[training_metric] + history.history[validation_metric])])
  plt.xlabel('Epoch')
  plt.ylabel(ylabel)
  plt.legend()
  plt.grid(True)
  plt.show()

In [ ]:
plotHistory(history, 'loss', 'val_loss', "Loss")
plotHistory(history, 'mae', 'val_mae', "Mean Absolute Error")

### Model evaluation

In [ ]:
# Computing the predictions of the test set
predictions = model.predict(normalized_test_features)

In [ ]:
# Function to compute the classification accuracy
def computeAccuracy():
    
    # Creating an array to save the results
    accuracy = np.zeros(len(naca_numbers))

    for idx in range(len(naca_numbers)):
        # Converting the NACA values to the closest interger
        naca_predictions = np.array([round(prediction) for prediction in predictions[:,idx]])
        naca_labels = np.array([round(label) for label in test_labels[:,idx]])

        # Extracting the samples correctly classified
        correctly_classified = np.where(np.equal(naca_predictions, naca_labels))

        # Computing the classification accuracy of the current NACA number
        accuracy[idx] = np.shape(correctly_classified)[1] / len(naca_labels)

    return accuracy

In [ ]:
# Computing the classification accuracy
accuracy = computeAccuracy()

# Computing the regression errors: MSE and MAE
loss, mae = model.evaluate(normalized_test_features, test_labels, verbose=0)

print("REGRESSION")
print(f" - Loss (Mean Square Error) --> {loss}")
print(f" - Mean Absolute Error --> {mae}")


print("\nCLASSIFICATION")
print(f" - Accuracy --> {np.mean(accuracy)}")
for i in range(len(naca_numbers)):
    print(f"   • {naca_numbers[i]} --> {accuracy[i]}")


In [ ]:
# Computing the predictions of the test set
predictions = model.predict(normalized_test_features)

In [ ]:
# Function to plot the predicted values
def plotPredictions(test_labels, test_predictions, label, color):
  plt.scatter(test_labels, test_predictions, label=label, color=color)
  plt.xlabel('True values')
  plt.ylabel('Predictions')
  plt.axis('equal')
  plt.axis('square')
  plt.xlim([0, np.max(test_labels)])
  plt.ylim([0, np.max(test_labels)])
  plt.plot([0, 100], [0, 100], color="black")
  plt.legend()

In [ ]:
colors = ["blue", "green", "orange"]

# Plotting the obtained results
for i in range(len(naca_numbers)):
  plotPredictions(test_labels[:,i], predictions[:,i], label=naca_numbers[i], color=colors[i])